Library

In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import random
from pathlib import Path
from tqdm.notebook import tqdm #import timm
from PIL import Image
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler  

Testing trunk

In [ ]:
class CFG:
    taxonomy_csv = "taxonomy.csv"         # species ID to label mapping
    train_datadir = "Denoise1"            # root of PNG folder
    TARGET_SHAPE = (256, 256)             # shape of spectrogram image
    aug_prob = 0.5                        # augmentation probability
    seed = 42
    debug = False                         # limit to 1000 samples if True
    LOAD_DATA = False                     # set to False to use spectrograms dict


# === Paths ===
train_csv = "train.csv"               # ← your BirdCLEF official file
spectrogram_root = Path("Denoise1")   # ← where your spectrogram PNGs are

# === Load train.csv and extract ogg stem names
df_train = pd.read_csv(train_csv)
df_train['ogg_stem'] = df_train['filename'].apply(lambda x: Path(x).stem)

# === Scan spectrograms and match with train.csv
records = []

# for folder in spectrogram_root.glob("*"):
#     if folder.is_dir():
#         for f in folder.glob("*.png"):
#             folder_id = folder.name
#             png_name = f.name
#             png_stem = f.stem.split("_seg")[0]  # e.g., iNat65519

#             match = df_train[df_train['ogg_stem'] == png_stem]
#             if not match.empty:
#                 row = match.iloc[0]
#                 records.append({
#                     "filename": f"{folder_id}/{png_name}",
#                     "primary_label": row["primary_label"],
#                     "secondary_labels": row.get("secondary_labels", "[]")
#                 })
#             else:
#                 print(f" No match found for {png_stem}")

# # === Save metadata.csv
# df_meta = pd.DataFrame(records)
# df_meta.to_csv("metadata_5s_denoise.csv", index=False)
# print(f" metadata_5s_denoise.csv saved with {len(df_meta)} rows.")

Configuration

In [37]:
class CFG:
    # === General Settings ===
    seed = 42
    debug = False                   # If True, limits dataset to 1000 samples
    apex = False
    print_freq = 100
    num_workers = 0
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    # === Paths ===
    taxonomy_csv = "taxonomy.csv"         # species ID to label mapping
    train_csv = "metadata_5s_denoise.csv"            # metadata linking PNG to label
    train_datadir = "Denoise1"            # directory containing PNG spectrograms

    # === Spectrogram Settings ===
    LOAD_DATA = False                     # False to load directly from PNG
    in_channels = 3                       # RGB PNG images
    TARGET_SHAPE = (256, 256)

    # === Audio Parameters (if needed elsewhere) ===
    FS = 32000
    TARGET_DURATION = 5.0
    N_FFT = 1024
    HOP_LENGTH = 512
    N_MELS = 128
    FMIN = 50
    FMAX = 14000

    # === Model and Training ===
    model_name = 'efficientnet_b0'
    pretrained = True
    epochs = 5
    batch_size = 32
    criterion = 'BCEWithLogitsLoss'
    optimizer = 'AdamW'
    lr = 5e-4
    weight_decay = 1e-5
    scheduler = 'CosineAnnealingLR'
    min_lr = 1e-6
    T_max = epochs

    # === Cross-Validation ===
    n_fold = 5
    selected_folds = [0, 1, 2, 3, 4]

    # === Augmentation ===
    aug_prob = 0.5
    mixup_alpha = 0.5

    # === Debug Shortcut ===
    def update_debug_settings(self):
        if self.debug:
            self.epochs = 2
            self.selected_folds = [0]

# Create config object
cfg = CFG()


Loading specturm from png

In [ ]:
class BirdCLEFDatasetFromPNG(Dataset):
    def __init__(self, df, cfg, mode="train"):
        self.df = df
        self.cfg = cfg
        self.mode = mode

        taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
        self.species_ids = taxonomy_df['primary_label'].tolist()
        self.num_classes = len(self.species_ids)
        self.label_to_idx = {label: idx for idx, label in enumerate(self.species_ids)}

        # Generate sample names and filepaths
        self.df['filepath'] = self.df['filename'].apply(lambda x: os.path.join(cfg.train_datadir, x))
        self.df['samplename'] = self.df['filename'].apply(lambda x: f"{x.split('/')[0]}-{Path(x).stem}")

        if cfg.debug:
            self.df = self.df.sample(min(1000, len(self.df)), random_state=cfg.seed).reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        filepath = row['filepath']
        samplename = row['samplename']
        

        try:
            img = Image.open(filepath).convert("RGB").resize(self.cfg.TARGET_SHAPE)
            spec = np.array(img).astype(np.float32) / 255.0  # Normalize to [0, 1]
            spec = torch.tensor(spec).permute(2, 0, 1)  # [C, H, W]
        except Exception as e:
            print(f"⚠️ Error loading image {filepath}: {e}")
            spec = torch.zeros(3, *self.cfg.TARGET_SHAPE)

        if self.mode == "train" and random.random() < self.cfg.aug_prob:
            spec = self.apply_spec_augmentations(spec)

        target = self.encode_label(row['primary_label'])

        if 'secondary_labels' in row and pd.notna(row['secondary_labels']) and row['secondary_labels'] not in [[''], None]:
            try:
                secondary_labels = eval(row['secondary_labels']) if isinstance(row['secondary_labels'], str) else row['secondary_labels']
                for label in secondary_labels:
                    if label in self.label_to_idx:
                        target[self.label_to_idx[label]] = 1.0
            except Exception as e:
                print(f"⚠️ Failed to parse secondary_labels for {samplename}: {e}")

        return {
            'melspec': spec,
            'target': torch.tensor(target, dtype=torch.float32),
            'filename': row['filename']
        }

    def apply_spec_augmentations(self, spec):
        if random.random() < 0.5:
            for _ in range(random.randint(1, 3)):
                width = random.randint(5, 20)
                start = random.randint(0, spec.shape[2] - width)
                spec[:, :, start:start + width] = 0

        if random.random() < 0.5:
            for _ in range(random.randint(1, 3)):
                height = random.randint(5, 20)
                start = random.randint(0, spec.shape[1] - height)
                spec[:, start:start + height, :] = 0

        if random.random() < 0.5:
            gain = random.uniform(0.8, 1.2)
            bias = random.uniform(-0.1, 0.1)
            spec = torch.clamp(spec * gain + bias, 0, 1)

        return spec

    def encode_label(self, label):
        target = np.zeros(self.num_classes, dtype=np.float32)
        if label in self.label_to_idx:
            target[self.label_to_idx[label]] = 1.0
        return target


def collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        return {}

    result = {key: [] for key in batch[0].keys()}

    for item in batch:
        for key, value in item.items():
            result[key].append(value)

    for key in result:
        if key == 'target':
            result[key] = torch.stack(result[key])
        elif key == 'melspec':
            shapes = [t.shape for t in result[key]]
            if len(set(str(s) for s in shapes)) == 1:
                result[key] = torch.stack(result[key])

    return result


Model building

In [ ]:
class BirdCLEFModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        
        taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
        cfg.num_classes = len(taxonomy_df)
        
        self.backbone = timm.create_model(
            cfg.model_name,
            pretrained=cfg.pretrained,
            in_chans=cfg.in_channels,
            drop_rate=0.2,
            drop_path_rate=0.2
        )
        
        if 'efficientnet' in cfg.model_name:
            backbone_out = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
        elif 'resnet' in cfg.model_name:
            backbone_out = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
        else:
            backbone_out = self.backbone.get_classifier().in_features
            self.backbone.reset_classifier(0, '')
        
        self.pooling = nn.AdaptiveAvgPool2d(1)
            
        self.feat_dim = backbone_out
        
        self.classifier = nn.Linear(backbone_out, cfg.num_classes)
        
        self.mixup_enabled = hasattr(cfg, 'mixup_alpha') and cfg.mixup_alpha > 0
        if self.mixup_enabled:
            self.mixup_alpha = cfg.mixup_alpha
            
    def forward(self, x, targets=None):
    
        if self.training and self.mixup_enabled and targets is not None:
            mixed_x, targets_a, targets_b, lam = self.mixup_data(x, targets)
            x = mixed_x
        else:
            targets_a, targets_b, lam = None, None, None
        
        features = self.backbone(x)
        
        if isinstance(features, dict):
            features = features['features']
            
        if len(features.shape) == 4:
            features = self.pooling(features)
            features = features.view(features.size(0), -1)
        
        logits = self.classifier(features)
        
        if self.training and self.mixup_enabled and targets is not None:
            loss = self.mixup_criterion(F.binary_cross_entropy_with_logits, 
                                       logits, targets_a, targets_b, lam)
            return logits, loss
            
        return logits
    
    def mixup_data(self, x, targets):
        """Applies mixup to the data batch"""
        batch_size = x.size(0)

        lam = np.random.beta(self.mixup_alpha, self.mixup_alpha)

        indices = torch.randperm(batch_size).to(x.device)

        mixed_x = lam * x + (1 - lam) * x[indices]
        
        return mixed_x, targets, targets[indices], lam
    
    def mixup_criterion(self, criterion, pred, y_a, y_b, lam):
        """Applies mixup to the loss function"""
        return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)



In [ ]:
def get_optimizer(model, cfg):
    """
    Create an optimizer for training.
    Chooses based on the option set in cfg.optimizer.
    """
    if cfg.optimizer == 'Adam':
        # Adam optimizer (adaptive learning rate)
        optimizer = optim.Adam(
            model.parameters(),
            lr=cfg.lr,
            weight_decay=cfg.weight_decay
        )
    elif cfg.optimizer == 'AdamW':
        # AdamW optimizer (better for weight decay regularization)
        optimizer = optim.AdamW(
            model.parameters(),
            lr=cfg.lr,
            weight_decay=cfg.weight_decay
        )
    elif cfg.optimizer == 'SGD':
        # Stochastic Gradient Descent with momentum
        optimizer = optim.SGD(
            model.parameters(),
            lr=cfg.lr,
            momentum=0.9,
            weight_decay=cfg.weight_decay
        )
    else:
        raise NotImplementedError(f"Optimizer {cfg.optimizer} is not supported.")

    return optimizer


def get_scheduler(optimizer, cfg):
    """
    Set the learning rate scheduler to adjust learning rate during training.
    Returns None if the scheduler is not needed or not implemented.
    """
    if cfg.scheduler == 'CosineAnnealingLR':
        # Slowly reduce learning rate following a cosine curve
        scheduler = lr_scheduler.CosineAnnealingLR(
            optimizer,
            T_max=cfg.T_max,
            eta_min=cfg.min_lr
        )
    elif cfg.scheduler == 'ReduceLROnPlateau':
        # Reduce learning rate when validation loss stops improving
        scheduler = lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=2,
            min_lr=cfg.min_lr,
            verbose=True
        )
    elif cfg.scheduler == 'StepLR':
        # Reduce learning rate by half after every few epochs
        scheduler = lr_scheduler.StepLR(
            optimizer,
            step_size=cfg.epochs // 3,
            gamma=0.5
        )
    elif cfg.scheduler == 'OneCycleLR':
        # OneCycle is created in the training loop instead
        scheduler = None
    else:
        # If no valid scheduler is specified
        scheduler = None

    return scheduler


def get_criterion(cfg):
    """
    Choose the loss function based on cfg.criterion.
    """
    if cfg.criterion == 'BCEWithLogitsLoss':
        # Binary Cross-Entropy with logits (used for multi-label classification)
        criterion = nn.BCEWithLogitsLoss()
    else:
        raise NotImplementedError(f"Loss function {cfg.criterion} is not supported.")

    return criterion


Loop

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device, scheduler=None):
    """
    Train the model for one full pass through the training data.
    Only uses 50% of the data in each epoch for faster training during development.
    """
    model.train()
    losses = []
    all_targets = []
    all_outputs = []

    data_iter = iter(loader)
    total_steps = int(0.5 * len(loader))  # Use only half of the training data

    pbar = tqdm(range(total_steps), total=total_steps, desc="Training")

    for step in pbar:
        try:
            batch = next(data_iter)
        except StopIteration:
            break  # No more data

        # Handle batches where melspectrograms are in a list format
        if isinstance(batch['melspec'], list):
            batch_outputs = []
            batch_losses = []

            for i in range(len(batch['melspec'])):
                inputs = batch['melspec'][i].unsqueeze(0).to(device)
                target = batch['target'][i].unsqueeze(0).to(device)

                optimizer.zero_grad()
                output = model(inputs)
                loss = criterion(output, target)
                loss.backward()

                batch_outputs.append(output.detach().cpu())
                batch_losses.append(loss.item())

            optimizer.step()
            outputs = torch.cat(batch_outputs, dim=0).numpy()
            loss = np.mean(batch_losses)
            targets = batch['target'].numpy()

        else:
            # Standard batch format
            inputs = batch['melspec'].to(device)
            targets = batch['target'].to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            # Some models may return (output, loss) together
            if isinstance(outputs, tuple):
                outputs, loss = outputs
            else:
                loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

            outputs = outputs.detach().cpu().numpy()
            targets = targets.detach().cpu().numpy()

        # Step the learning rate scheduler if it’s being used
        if scheduler is not None and isinstance(scheduler, lr_scheduler.OneCycleLR):
            scheduler.step()

        all_outputs.append(outputs)
        all_targets.append(targets)
        losses.append(loss if isinstance(loss, float) else loss.item())

        pbar.set_postfix({
            'train_loss': np.mean(losses[-10:]) if losses else 0,
            'lr': optimizer.param_groups[0]['lr']
        })

    # Combine all outputs and targets and calculate average loss and AUC
    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)
    auc = calculate_auc(all_targets, all_outputs)
    avg_loss = np.mean(losses)

    return avg_loss, auc


def validate(model, loader, criterion, device):
    """
    Run the model on validation data (no gradient updates).
    Returns loss and AUC for evaluation.
    """
    model.eval()
    losses = []
    all_targets = []
    all_outputs = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            # Handle individual spectrograms (list format)
            if isinstance(batch['melspec'], list):
                batch_outputs = []
                batch_losses = []

                for i in range(len(batch['melspec'])):
                    inputs = batch['melspec'][i].unsqueeze(0).to(device)
                    target = batch['target'][i].unsqueeze(0).to(device)

                    output = model(inputs)
                    loss = criterion(output, target)

                    batch_outputs.append(output.detach().cpu())
                    batch_losses.append(loss.item())

                outputs = torch.cat(batch_outputs, dim=0).numpy()
                loss = np.mean(batch_losses)
                targets = batch['target'].numpy()

            else:
                inputs = batch['melspec'].to(device)
                targets = batch['target'].to(device)

                outputs = model(inputs)
                loss = criterion(outputs, targets)

                outputs = outputs.detach().cpu().numpy()
                targets = targets.detach().cpu().numpy()

            all_outputs.append(outputs)
            all_targets.append(targets)
            losses.append(loss if isinstance(loss, float) else loss.item())

    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)

    auc = calculate_auc(all_targets, all_outputs)
    avg_loss = np.mean(losses)

    return avg_loss, auc


def calculate_auc(targets, outputs):
    """
    Compute AUC (Area Under Curve) score for each class and return the average.
    Only includes classes that have at least one positive example.
    """
    num_classes = targets.shape[1]
    aucs = []

    # Convert logits to probabilities
    probs = 1 / (1 + np.exp(-outputs))

    for i in range(num_classes):
        if np.sum(targets[:, i]) > 0:  # Skip if the class has no positives
            class_auc = roc_auc_score(targets[:, i], probs[:, i])
            aucs.append(class_auc)

    return np.mean(aucs) if aucs else 0.0


In [ ]:
def run_training(df, cfg):
    """Train the model using spectrogram images (either precomputed or created during training)."""

    # Load list of bird species from taxonomy file
    taxonomy_df = pd.read_csv(cfg.taxonomy_csv)
    species_ids = taxonomy_df['primary_label'].tolist()
    cfg.num_classes = len(species_ids)

    # Apply debug settings if needed
    if cfg.debug:
        cfg.update_debug_settings()

    # Try to load precomputed spectrograms
    spectrograms = None
    if cfg.LOAD_DATA:
        print("Trying to load saved mel spectrograms...")
        try:
            spectrograms = np.load(cfg.spectrogram_npy, allow_pickle=True).item()
            print(f"Loaded {len(spectrograms)} spectrograms.")
        except Exception as e:
            print(f"Could not load spectrograms: {e}")
            print("Will generate spectrograms during training instead.")
            cfg.LOAD_DATA = False

    # If not loading, prepare to generate during training
    if not cfg.LOAD_DATA:
        print("Spectrograms will be created during training.")
        
        # Check for required columns
        if 'filename' not in df.columns:
            if 'filepath' in df.columns:
                df['filename'] = df['filepath'].apply(lambda x: os.path.relpath(x, cfg.train_datadir))
            else:
                raise ValueError("Metadata must contain either 'filename' or 'filepath' column.")

        # Create file path and sample name
        df['filepath'] = df['filename'].apply(lambda x: os.path.join(cfg.train_datadir, x))
        if 'samplename' not in df.columns:
            df['samplename'] = df['filename'].apply(
                lambda x: f"{x.split('/')[0]}-{os.path.splitext(os.path.basename(x))[0]}"
            )

    # Set up cross-validation
    skf = StratifiedKFold(n_splits=cfg.n_fold, shuffle=True, random_state=cfg.seed)
    best_scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['primary_label'])):
        if fold not in cfg.selected_folds:
            continue

        print(f'\n{"="*30} Fold {fold} {"="*30}')

        # Split data
        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)

        print(f'Training samples: {len(train_df)}')
        print(f'Validation samples: {len(val_df)}')

        # Create datasets and data loaders
        train_dataset = BirdCLEFDatasetFromPNG(train_df, cfg, mode='train')
        val_dataset = BirdCLEFDatasetFromPNG(val_df, cfg, mode='valid')

        train_loader = DataLoader(
            train_dataset, batch_size=cfg.batch_size, shuffle=True,
            num_workers=cfg.num_workers, pin_memory=True,
            collate_fn=collate_fn, drop_last=True
        )
        val_loader = DataLoader(
            val_dataset, batch_size=cfg.batch_size, shuffle=False,
            num_workers=cfg.num_workers, pin_memory=True,
            collate_fn=collate_fn
        )

        # Build model and training components
        model = BirdCLEFModel(cfg).to(cfg.device)
        optimizer = get_optimizer(model, cfg)
        criterion = get_criterion(cfg)

        # Set up learning rate scheduler
        if cfg.scheduler == 'OneCycleLR':
            scheduler = lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=cfg.lr,
                steps_per_epoch=len(train_loader),
                epochs=cfg.epochs,
                pct_start=0.1
            )
        else:
            scheduler = get_scheduler(optimizer, cfg)

        # Start training loop
        best_auc = 0
        best_epoch = 0

        for epoch in range(cfg.epochs):
            print(f"\nEpoch {epoch+1} of {cfg.epochs}")

            # Train for one epoch
            train_loss, train_auc = train_one_epoch(
                model, train_loader, optimizer, criterion, cfg.device,
                scheduler if isinstance(scheduler, lr_scheduler.OneCycleLR) else None
            )

            # Validate model
            val_loss, val_auc = validate(model, val_loader, criterion, cfg.device)

            # Step the scheduler
            if scheduler and not isinstance(scheduler, lr_scheduler.OneCycleLR):
                if isinstance(scheduler, lr_scheduler.ReduceLROnPlateau):
                    scheduler.step(val_loss)
                else:
                    scheduler.step()

            print(f"Train Loss: {train_loss:.4f}, AUC: {train_auc:.4f}")
            print(f"Val Loss: {val_loss:.4f}, AUC: {val_auc:.4f}")

            # Save model if it's the best so far
            if val_auc > best_auc:
                best_auc = val_auc
                best_epoch = epoch + 1
                print(f"New best AUC: {best_auc:.4f} at epoch {best_epoch}")
                torch.save(model.state_dict(), f"model_fold{fold}.pth")

        # Store best score
        best_scores.append(best_auc)
        print(f"\nBest AUC for fold {fold}: {best_auc:.4f} at epoch {best_epoch}")

        # Clear memory
        del model, optimizer, scheduler, train_loader, val_loader
        torch.cuda.empty_cache()
        gc.collect()

    # Print overall results
    print("\n" + "="*60)
    print("Cross-Validation Results:")
    for i, score in enumerate(best_scores):
        print(f"Fold {cfg.selected_folds[i]}: AUC = {score:.4f}")
    print(f"Average AUC: {np.mean(best_scores):.4f}")
    print("="*60)


In [44]:
if __name__ == "__main__":
    import time
    import shutil

    print("\nLoading training data...")
    train_df = pd.read_csv(cfg.train_csv)
    taxonomy_df = pd.read_csv(cfg.taxonomy_csv)

    print("\nStarting training...")
    print(f"LOAD_DATA is set to {cfg.LOAD_DATA}")
    if cfg.LOAD_DATA:
        print("Using pre-computed mel spectrograms from NPY file")
    else:
        print("Will generate spectrograms on-the-fly during training")

    run_training(train_df, cfg)

    # Optional: save the best fold model as "best_model.pth"
    best_fold = cfg.selected_folds[0]  # You could change logic here
    best_model_path = f"model_fold{best_fold}.pth"
    if os.path.exists(best_model_path):
        shutil.copy(best_model_path, "best_model.pth")
        print(f"\n✅ Saved best model to: best_model.pth (copied from model_fold{best_fold}.pth)")

    print("\nTraining complete!")



Loading training data...

Starting training...
LOAD_DATA is set to False
Will generate spectrograms on-the-fly during training
Will generate spectrograms on-the-fly during training.


e:\Anaconda\envs\torch-cuda\lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(



============================== Fold 0 ==============================
Training set: 130628 samples
Validation set: 32657 samples

Epoch 1/5


Training:   0%|          | 0/2041 [00:00<?, ?it/s]

Validation:   0%|          | 0/1021 [00:00<?, ?it/s]

...

Validation:   0%|          | 0/1021 [00:00<?, ?it/s]

Train Loss: 0.0199, Train AUC: 0.9314
Val Loss: 0.0186, Val AUC: 0.9446
New best AUC: 0.9446 at epoch 4

Epoch 5/5


Training:   0%|          | 0/2041 [00:00<?, ?it/s]

Validation:   0%|          | 0/1021 [00:00<?, ?it/s]

Train Loss: 0.0184, Train AUC: 0.9473
Val Loss: 0.0178, Val AUC: 0.9494
New best AUC: 0.9494 at epoch 5

Best AUC for fold 4: 0.9494 at epoch 5

Cross-Validation Results:
Fold 0: 0.9433
Fold 1: 0.9470
Fold 2: 0.9473
Fold 3: 0.9493
Fold 4: 0.9494
Mean AUC: 0.9473

✅ Saved best model to: best_model.pth (copied from model_fold0.pth)

Training complete!
